In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from StringIO import StringIO
import requests

In [39]:
#load in data
data_path='../data_extraction/data/'

cohort = pd.read_table(data_path+'cohort.tsv')
creatinine = pd.read_table(data_path+'creatinine.csv')
maps = pd.read_table(data_path+'map.csv')
urine = pd.read_table(data_path+'urine.csv')

In [40]:
#numerical variable summaries
print cohort[['age','los']].describe()

               age          los
count  5342.000000  5342.000000
mean     71.703842    11.416165
std      48.474031    10.331876
min      18.138720     3.000300
25%      52.494296     4.790000
50%      65.322301     7.903400
75%      76.984900    14.118000
max     309.777981   173.072500


In [41]:
#gender summary
print pd.value_counts(cohort['gender'].values, sort=False)

F    2613
M    2729
dtype: int64


In [146]:
#ethnicity summary
eth = cohort['ethnicity'].astype('category')

#grouping based on the standards at: http://grants.nih.gov/grants/guide/notice-files/NOT-OD-15-089.html
eth = eth.cat.add_categories(['HISPANIC/LATINO','MULTI/OTHER','UNKNOWN'])

eth[np.array([('HISPANIC' in i or
               'PORTUGUESE' in i) for i in eth],dtype=bool)]='HISPANIC/LATINO'
eth[np.array([('ASIAN' in i) for i in eth],dtype=bool)]='ASIAN'
eth[np.array([('BLACK' in i or
               'AFRICAN' in i) for i in eth],dtype=bool)]='BLACK/AFRICAN AMERICAN'
eth[np.array([('WHITE' in i or
               'MIDDLE EAST' in i) for i in eth],dtype=bool)]='WHITE'
eth[np.array([('MULTI' in i or
               'OTHER' in i) for i in eth],dtype=bool)]='MULTI/OTHER'
eth[np.array([('DECLINE' in i or
               'UNABLE' in i or
               'UNKNOWN' in i) for i in eth],dtype=bool)]='UNKNOWN'

eth = eth.cat.remove_unused_categories()

print pd.value_counts(eth.values, sort=False)

AMERICAN INDIAN/ALASKA NATIVE       2
ASIAN                             105
BLACK/AFRICAN AMERICAN            346
WHITE                            3885
HISPANIC/LATINO                   169
MULTI/OTHER                       146
UNKNOWN                           689
dtype: int64
